In [1]:
import transformers
import os
import pandas as pd
import natsort
import numpy as np
from PIL import Image
import time

import nltk
import numpy as np
import pandas as pd
import transformers
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
from gensim.models import Word2Vec, word2vec
from transformers import BertTokenizer, BertModel, BertConfig, AutoModel, AdamW
import torch
from transformers import AutoTokenizer, DistilBertModel
from PIL import Image
import natsort
import os

## Import/Load Dependencies:
# timm==0.4.12
# transformers==4.15.0  #using it as replacment of 4.22.0
# fairscale==0.4.4
# pycocoevalcap

if os.getcwd() != "/home/nitesh/Documents/MY_THESIS/MAMI/BLIP":
    %cd BLIP
print(os.getcwd())

print('transformer version:', transformers.__version__)
# !git clone https://github.com/salesforce/BLIP
# Ref: https://github.com/salesforce/BLIP

[nltk_data] Downloading package stopwords to /home/nitesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-06-03 14:02:12.627597: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


/home/nitesh/Documents/MY_THESIS/MAMI/BLIP
/home/nitesh/Documents/MY_THESIS/MAMI/BLIP
transformer version: 4.15.0


In [2]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Create the directory if it doesn't exist

model_directory = "./DistilBERT_Caption_EmbModel_&_Tokenizer/DistilBERT_Model"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)

tokenizer_directory = "./DistilBERT_Caption_EmbModel_&_Tokenizer/DistilBERT_tokenizer"
if not os.path.exists(tokenizer_directory):
    os.makedirs(tokenizer_directory)
    
    
model.save_pretrained(model_directory)
tokenizer.save_pretrained(tokenizer_directory)

('./DistilBERT_Caption_EmbModel_&_Tokenizer/DistilBERT_tokenizer/tokenizer_config.json',
 './DistilBERT_Caption_EmbModel_&_Tokenizer/DistilBERT_tokenizer/special_tokens_map.json',
 './DistilBERT_Caption_EmbModel_&_Tokenizer/DistilBERT_tokenizer/vocab.txt',
 './DistilBERT_Caption_EmbModel_&_Tokenizer/DistilBERT_tokenizer/added_tokens.json')

In [4]:
device = "cpu"
model.to(device)
projection_layer = torch.nn.Linear(768, 64)  # Add a linear layer to project embeddings to size 64
projection_layer.to(device)

Linear(in_features=768, out_features=64, bias=True)

In [5]:
# Load the 
train = pd.read_csv('train_BLIP_captions.csv')

test = pd.read_csv('test_BLIP_captions.csv')



In [10]:
train_encodings = tokenizer(
    train["text"].tolist(),
    padding=True,
    max_length=20,
    truncation=True,
    return_tensors="pt"
)

test_encodings = tokenizer(
    test["text"].tolist(),
    padding=True,
    max_length=20,
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    train_outputs = model(**train_encodings.to(device))
    train_embeddings = train_outputs.last_hidden_state[:, 0, :]  # Extract embeddings for the [CLS] token
    train_embeddings = projection_layer(train_embeddings)  # Project embeddings to size 65
    
    test_outputs = model(**test_encodings.to(device))
    test_embeddings = test_outputs.last_hidden_state[:, 0, :]  # Extract embeddings for the [CLS] token
    test_embeddings = projection_layer(test_embeddings)  # Project embeddings to size 65

In [11]:
print("train embeddings shape:", train_embeddings.shape)
print("test embeddings shape:", test_embeddings.shape)

train embeddings shape: torch.Size([9986, 64])
test embeddings shape: torch.Size([1000, 64])


In [12]:
train_embeddings

tensor([[-0.1932,  0.0834, -0.7582,  ..., -0.4121,  0.2424, -0.0134],
        [-0.2080,  0.0656, -0.7333,  ..., -0.3051,  0.0820, -0.0117],
        [-0.1806,  0.1146, -0.6511,  ..., -0.2794,  0.2367, -0.0113],
        ...,
        [-0.1549,  0.1752, -0.6844,  ..., -0.3982,  0.2638, -0.0273],
        [-0.0293,  0.1082, -0.6396,  ..., -0.4435,  0.1650,  0.0063],
        [-0.1633,  0.0984, -0.7528,  ..., -0.3874,  0.1956, -0.0389]])

In [13]:
test_embeddings

tensor([[-0.1756,  0.0793, -0.7914,  ..., -0.3138,  0.2691, -0.0357],
        [-0.2032,  0.0948, -0.5722,  ..., -0.3676,  0.2403, -0.1027],
        [-0.2120,  0.1770, -0.6337,  ..., -0.3392,  0.3496, -0.1244],
        ...,
        [-0.2588,  0.1198, -0.7081,  ..., -0.4302,  0.2636, -0.0382],
        [-0.1236,  0.1029, -0.6533,  ..., -0.3789,  0.2802,  0.0214],
        [-0.1453,  0.1984, -0.7111,  ..., -0.4090,  0.1909,  0.0190]])